<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-разделение-данных" data-toc-modified-id="Загрузка-и-разделение-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и разделение данных</a></span></li><li><span><a href="#Названия-столбцов" data-toc-modified-id="Названия-столбцов-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Названия столбцов</a></span></li><li><span><a href="#Цикл-CatBoost" data-toc-modified-id="Цикл-CatBoost-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Цикл CatBoost</a></span></li></ul></div>

## Загрузка и разделение данных

In [1]:
import pandas as pd
import pyarrow as pa
import numpy as np
import pyarrow.csv as csv
import pyarrow.parquet as pq
from tqdm import tqdm
from catboost import CatBoostClassifier
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
X_train = pq.read_table('./datasets/X_train.parquet').to_pandas()

In [4]:
y_train = pq.read_table('./datasets/y_train.parquet').to_pandas()

In [5]:
X_test = pq.read_table('./datasets/X_test.parquet').to_pandas()

## Названия столбцов

In [6]:
exhausters_X_list = ['ЭКСГАУСТЕР 4', 'ЭКСГАУСТЕР 5', 'ЭКСГАУСТЕР 6', 'ЭКСГАУСТЕР 7', 'ЭКСГАУСТЕР 8', 'ЭКСГАУСТЕР 9']
exhausters_X_list

['ЭКСГАУСТЕР 4',
 'ЭКСГАУСТЕР 5',
 'ЭКСГАУСТЕР 6',
 'ЭКСГАУСТЕР 7',
 'ЭКСГАУСТЕР 8',
 'ЭКСГАУСТЕР 9']

In [7]:
exhausters_y_list = ['ЭКСГАУСТЕР А/М №4' ,'ЭКСГАУСТЕР А/М №5', 'ЭКСГАУСТЕР А/М №6', 'ЭКСГАУСТЕР А/М №7', 'ЭКСГАУСТЕР А/М №8', 'ЭКСГАУСТЕР А/М №9']
exhausters_y_list

['ЭКСГАУСТЕР А/М №4',
 'ЭКСГАУСТЕР А/М №5',
 'ЭКСГАУСТЕР А/М №6',
 'ЭКСГАУСТЕР А/М №7',
 'ЭКСГАУСТЕР А/М №8',
 'ЭКСГАУСТЕР А/М №9']

## Цикл CatBoost

In [ ]:
### exhausters cycle for M3
result = {}
for ex_X, ex_y in tqdm(zip(exhausters_X_list, exhausters_y_list)):
    X_columns = [col for col in X_train.columns if ex_X in col]
    y_columns = [col for col in y_train.columns if ex_y in col]
    features_train = X_train[X_columns]
    targets_train = y_train[y_columns]
    features_test = X_test[X_columns]
    # добавляем  .rolling(100).std(), diff(1), rolling(100).median(), row - row.mean(), row - row.median()
    for feature in features_train.columns:
        name = feature+'r100.std'
        features_train[name] = features_train[feature].rolling(100).std()
        features_test[name] = features_test[feature].rolling(100).std()
        name = feature+'diff1'
        features_train[name] = features_train[feature].diff(1)
        features_test[name] = features_test[feature].diff(1)
        name = feature+'r100.mean'
        features_train[name] = features_train[feature].rolling(100).median()
        features_test[name] = features_test[feature].rolling(100).median()
        name = feature+'-mean'
        features_train[name] = features_train[feature] - features_train[feature].mean() 
        features_test[name] = features_test[feature] - features_test[feature].mean() 
        name = feature+'-median'
        features_train[name] = features_train[feature] - features_train[feature].median()
        features_test[name] = features_test[feature] - features_test[feature].median()
    for target in targets_train.columns:
        # убираем метку 1 (останов)
        targets_train.loc[targets_train[target] == 1, target] = 0
        targets_train.loc[targets_train[target] == 2, target] = 1
        if len(targets_train[target].unique()) == 2:
            cbc = CatBoostClassifier(task_type='GPU',
                                random_seed=12345,
                                iterations=1000,
                                verbose=500,
                                auto_class_weights='Balanced')
            cbc.fit(features_train, targets_train[target])
            predictions = cbc.predict(features_test)*2
        else:
            predictions = np.array([targets_train[target].unique()]*len(features_test))
        result[target] = predictions
result = pd.DataFrame(result, index=X_test.index)

0it [00:00, ?it/s]